---
title: "Mag7Alpha"
format:
  html:
    engine: jupyter
execute:
  echo: true
  eval: true
jupyter: python3
---




### **Strategy Overview** {.test}
Market makers are large financial institutions who act as intermediaries between buyers and sellers. They provide liquidity to traders, taking orders on both sides and reducing the difference between ask and bid price for assets. Market makers make profit from bid ask spreads.
Our strategy is market-making on one of the Mag7 stocks, which includes Alphabet, Amazon, Apple, Meta, Microsoft, NVDIA, and Tesla. For our example, we will do MSFT. We will use the Stoikov market-making strategy, so we need two main equations:
The first calculates the reservation price based on the equation:

![Reservation Price](Equation.png)

s = current market mid price
q = quantity of assets in inventory of base asset (could be positive/negative for long/short positions)
σ = market volatility
T = closing time, when the measurement period ends (conveniently normalized to 1)
t = current time (T is normalized = 1, so t is a time fraction)

The second sets optimal bid-ask spread using the equation:

![Optimal Bid-Ask Spread](BidaskSpread.png)

σ = market volatility
T = closing time, when the measurement period ends (conveniently normalized to 1)
t = current time (T is normalized = 1, so t is a time fraction)
δa, δb = bid/ask spread, symmetrical → δa=δb
γ = inventory risk aversion parameter
κ = order book liquidity parameter

Entry conditions:
We want to create symmetrical bid and ask orders around the market mid-price, but this could lead to the inventory skewing in one direction if there are significant market movements in one direction. The reference price is where the buy and sell orders will be created around.

After calculating reservation price and optimal bid ask spreads:
Bid offer price = reservation price — optimal spread / 2
Ask offer price = reservation price + optimal spread / 2

We then enter into limit orders on both sides of this quote.


Exit conditions:
As the trading day goes on, each parameter of the models will change, and new values for reservation price and optimal spreads will be calculated. We then  set new orders based on the new parameters. This cycle continues indefinitely until the end of our backtesting period.

Stop Loss:
Instead of setting a specific stop loss, we set inventory limits to try and control the portfolio during momentum shifts. We set this at 15.

Data:
We can use ShinyBroker to obtain most of the assets’ attributes, such as price throughout the day, volatility, and timing, via fetch_historical_data.


### Blotter:


In [ ]:
#| echo: false
import pandas as pd
from itables import show
import itables.options as opt

opt.mode = "inline"

# Read the blotter CSV
blotter = pd.read_csv("blotter.csv")

show(blotter)

### Ledger:


In [ ]:
#| echo: false
import pandas as pd
from itables import show
import itables.options as opt

opt.mode = "inline"

# Read the ledger CSV
ledger = pd.read_csv("ledger.csv")

show(ledger)

### Portfolio Value Over Time


In [ ]:
#| echo: false
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "iframe_connected"

# Load ledger
ledger = pd.read_csv("ledger.csv")
ledger['timestamp'] = pd.to_datetime(ledger['timestamp'])

# Create line chart of portfolio value
fig = px.line(
    ledger,
    x='timestamp',
    y='portfolio_value',
    title='Portfolio Value Over Time',
    labels={'timestamp': 'Time', 'portfolio_value': 'Portfolio Value ($)'},
    template='plotly_white'
)
fig.update_layout(
    yaxis_tickformat=',',  # Adds commas: e.g., 100,000
)
fig.show()

### Inventory Level Over Time


In [ ]:
#| echo: false
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "iframe_connected"

# Load data
ledger = pd.read_csv("ledger.csv")
ledger['timestamp'] = pd.to_datetime(ledger['timestamp'])

# Create chart
fig = px.line(
    ledger,
    x='timestamp',
    y='inventory',
    title='Inventory Over Time',
    labels={'inventory': 'Inventory Level', 'timestamp': 'Time'},
    template='plotly_white'
)

# Show chart
fig.show()